## Created by yunsuxiaozi 2024/3/26

### 在这个notebook中,我们将介绍朴素贝叶斯算法的数学原理和代码实现,并使用Kaggle上的数据集来测试算法的效果,最后和开源库的效果进行比较.

## 数学原理:

### 首先是$P(A|B)P(B)=P(B|A)P(A)$,直接看公式可能不舒服,我解释一下.$P(B)$是B事件发生的概率,$P(A|B)$是B事件发生的情况下A事件发生的概率.所以$P(A|B)P(B)$是A,B事件同时发生的概率.后半部分也是A,B事件同时发生的概率,故相等.

### 我们可以将这个公式变成:$P(A|B)=\frac{P(B|A)P(A)}{P(B)}$.

### 我们一般用朴素贝叶斯是做分类任务的,比如我们想知道当$X=X0$时,$y=y0$的概率是多大,用公式表示就是$P(y=y0|X=X0)=\frac{P(X=X0|y=y0)P(y=y0)}{P(X=X0)}$

### 这里我们$X$的每个特征(每列)都是类别型变量,同时朴素贝叶斯是假设每个特征都是独立的,即:$P(X0[0],X0[1],……,X0[n]|y0)=P(X0[0]|y=y0)P(X0[1]|y0)……P(X0[n]|y=y0)$

### 所以,$P(y=y0|X=X0)=\frac{P(X0[0]|y=y0)P(X0[1]|y=y0)……P(X0[n]|y=y0)P(y=y0)}{P(X=X0)}$.这就是朴素贝叶斯分类器的原理.

### 接下来来仔细研究一下公式每部分:

- 分母是P(X=X0).这个其实是个定值,因为当训练数据确定,比如总共有2个特征,第1个特征2个类别,第2个特征3个类别,那么X的取值就有6种,X0无论是其中任意一种,$P(X=X0)=\frac{1}{6}$.由于这个值是定值,所以在算法实现上可以去掉.

- 分子中P(y=y0):这个其实就是y取某个类别的概率,这个取决于训练数据target的数据分布.

- 分子中P(X0[0]|y=y0)P(X0[1]|y=y0)……P(X0[n]|y=y0):其实就是在y=y0时那些数据中X的第i个特征取到X0[i]的概率.

### 在算法的实现上,P(X=X0)这部分可以去掉,还需要考虑到预测概率的归一化.最后实现算法的时候是P(y=y0|X=X0):= P(X0[0]|y=y0)P(X0[1]|y=y0)……P(X0[n]|y=y0)P(y=y0),由于是一堆概率相乘,而概率值又是0到1之间的数,所以之后的概率肯定会非常小,我们在做多分类任务的时候要考虑将概率值进行归一化,使它最终的预测概率求和等于1.这里用P/sum(P)来进行归一化.

### 下面我们通过Kaggle上<a href="https://www.kaggle.com/competitions/nlp-getting-started">入门自然语言处理的比赛数据</a>来实现算法,并和python现有库进行比较.

### 导入必要的python库,并固定随机种子,保证模型可以复现.

In [1]:
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import re#用于正则表达式提取
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

import random#提供了一些用于生成随机数的函数
class Config():
    seed=2024
    word_count=1000#统计出现最多的几个词
    train_path="/kaggle/input/nlp-getting-started/train.csv"
    test_path="/kaggle/input/nlp-getting-started/test.csv"
#设置随机种子,保证模型可以复现
def seed_everything(seed):
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(seed=Config.seed)

### 这里是导入训练数据,并对数据进行了一些清洗.

In [2]:
%time
import nltk#Natural Language Tokens 强大的自然语言处理库
from nltk.corpus import stopwords#导入英语的停用词表
import re#正则表达式操作的库
import emoji#处理字符串中的表情符号
#去除html标签的函数
def removeHTML(x):
    #用正则表达式提取例如<p>,<div>之类的标签
    html=re.compile(r'<.*?>')
    #用空字符串去替代这些标签.
    return html.sub(r'',x)
#对文本数据进行数据预处理,去除对于情感分析没有用的词.
def dataPreprocessing(x):  
    #导入英文的停用词
    stopword = stopwords.words('english')#导入英文的停用词
    #将字符串转成小写字母
    x = x.lower()
    #将字符串去除html标签
    x = removeHTML(x)
    #将文本中的表情符号（emoji）转换为文本文本形式。例如，将表示笑脸的" "表情符号转换为":face_with_tears_of_joy:"。delimiters=(" ", " ")是分隔符.
    x = emoji.demojize(x, delimiters=(" ", " "))
    #将评论中“@匀速小子”之类的字母、数字或下划线(\w)替换成空格.
    x = re.sub("@\w+", '',x)
    #删除评论中带单引号的数字
    x = re.sub("'\d+", '',x)
    #删除评论中所有的数字
    x = re.sub("\d+", '',x)
    #删除评论中的标点符号和特征字符,将不是(^)字母、数字或下划线(\w)或者空格(\s)的字符替换成空格.
    x = re.sub(r"[^\w\s]", '',x)
    #删除url:http格式的
    x = re.sub("http\w+", '',x)
    #将单个的小写字母替换成空格
    x = re.sub("\s[a-z]\s", '',x)
    #删除字符串开头和结尾的空格字符
    x = x.strip()
    #将文本分割成单词或者标点符号（tokenization）
    tokens=nltk.word_tokenize(x)
    tokens=[token for token in tokens if token not in stopword] 
    #返回处理好的字符串
    return tokens
train_df=pd.read_csv(Config.train_path)
train_df['text']=train_df['text'].apply(dataPreprocessing)
print(f"len(train_df):{len(train_df)}")

test_df=pd.read_csv(Config.test_path)
test_df['text']=test_df['text'].apply(dataPreprocessing)
print(f"len(test_df):{len(test_df)}")
test_df.head()

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.82 µs
len(train_df):7613
len(test_df):3263


,id,keyword,location,text
0,0,NaN,NaN,"[happenedterrible, car, crash]"
1,2,NaN,NaN,"[heard, earthquake, different, cities, stay, s..."
2,3,NaN,NaN,"[isforest, fire, spot, pond, geese, fleeing, a..."
3,9,NaN,NaN,"[apocalypse, lighting, spokane, wildfires]"
4,11,NaN,NaN,"[typhoon, soudelor, kills, china, taiwan]"


### 这里将训练数据的文本统计成字典,并选择出现次数最多的1000个词.

In [3]:
text=train_df['text'].values
word_dict={}
for i in range(len(text)):
    for j in range(len(text[i])):
        if text[i][j] in word_dict:
            word_dict[text[i][j]]+=1
        else:
            word_dict[text[i][j]]=1
#word_dict.items() 是key和value,sorted是排序,按照-value按照从小到大排序
sorted_items = sorted(word_dict.items(), key=lambda x: -x[1])

sorted_list = list(sorted_items)
top_words=[value[0] for value in sorted_list[:Config.word_count]]
top_words[:10]

['amp', 'im', 'like', 'fire', 'via', 'new', 'news', 'get', 'one', 'people']

### 构造训练数据和测试数据

In [4]:
train_text=train_df['text'].values
test_text=test_df['text'].values
X=np.array([[int(top_word in text) for top_word in top_words] for text in train_text])
y=train_df['target'].values
test_X=np.array([[int(top_word in text) for top_word in top_words] for text in test_text])
print(f"X.shape:{X.shape},y.shape:{y.shape}")

X.shape:(7613, 1000),y.shape:(7613,)


### 这里是我自己手写的朴素贝叶斯算法.

In [5]:
#朴素贝叶斯分类算法
class NaiveBayesClassifier():
    
    def __init__(self,num_classes=2):
        self.num_classes=num_classes#我们假设是二分类任务
        self.Pa=np.ones(self.num_classes)/self.num_classes#初始化每个类别概率相等,后续需要考虑target的分布.
        self.target=np.arange(self.num_classes)#分类的几种类别是什么
        self.features=[]#X每列分别有哪些类别
        self.featureP=[]#y=target[i]时X的j列为第K种类别的概率
        self.eps=1e-15#防止被除数为0
    def fit(self,train_X,train_y):
        self.target=sorted(np.unique(train_y))#训练数据y中出现几种类别
        self.num_classes=len(self.target)#类别数为多少
        feature=train_X.T#第i行就是第i个特征
        #统计每个特征有哪几种类别
        for i in range(len(feature)):
            self.features.append(np.unique(feature[i]))

        #用来统计y为每个类别的概率
        Pa=[]
        for i in range(len(self.target)):
            Pa.append(np.mean(train_y==self.target[i]))
        self.Pa=Pa
        
        #统计y=target[i]时每个特征每个类别的概率
        for i in range(len(self.target)):
            #当y=target[i]的时候的训练数据
            target_X=train_X[np.where((y==self.target[i]))[0]]
            
            #统计所有特征每个类别的概率
            feature_X=target_X.T
            featurePs=[]#y=target时X每种概率的分布情况
            for j in range(len(feature_X)):#第j个feature有这几种类别:self.features[j]
                
                #统计train_X中第j个特征每个类别的概率
                featureP=[]
                for k in range(len(self.features[j])):
                    featureP.append(np.mean(feature_X[j]==self.features[j][k]))
                
                featurePs.append(featureP)
            self.featureP.append(featurePs)
            
    def predict_proba(self,test_X):
        #每个数据每个类别的概率
        test_pros=np.zeros((len(test_X),self.num_classes))
        
        for i in range(len(test_X)):#test_X[i]是其中一个数据
            #数据text_X[i]统计每个类别的概率:P(X0[0]|y=y0)P(X0[1]|y=y0)……P(X0[n]|y=y0)P(y=y0)
            test_pro=np.zeros((self.num_classes))
            for j in range(self.num_classes):
                init_p=self.Pa[j]#P(y=y0)
                #找到text_X的第i个数据在y=target[j]时第k个特征的概率
                for k in range(len(test_X[i])):
                    Pba=self.Pa[j]
                    #找到第test_X的第i个数据的第K个特征是什么?
                    for l in range(len(self.features[k])):
                        if test_X[i][k]==self.features[k][l]:
                            Pba=self.featureP[j][k][l]      
                    init_p*=Pba
                test_pro[j]=init_p
            test_pros[i]=test_pro
        #最后对预测的概率进行归一化的操作
        test_pros=test_pros/ (np.sum(test_pros,axis=1).reshape(-1,1)+self.eps)
        return test_pros
 
    def predict(self,test_X):
        test_pros=self.predict_proba(test_X)
        test_preds=np.argmax(test_pros,axis=1)
        return test_preds

### 如果单看训练数据,准确率达到了0.81,其实在测试数据上还是有0.78的准确率.

In [6]:
def accuracy(y_true,y_pred):
    return np.mean(y_true==y_pred)
model=NaiveBayesClassifier()
model.fit(X,y)
y_pred=model.predict(X)
print(f"accuracy:{accuracy(y_pred,y)}")
test_pred=model.predict(test_X)

accuracy:0.8122947589649284


### 我们也可以看看python开源库的朴素贝叶斯分类器,效果比我的算法略好一点.

In [7]:
#https://www.kaggle.com/code/hadriencr/ml-olympiad-naivebayesclassifier
from sklearn.naive_bayes import CategoricalNB

classifier = CategoricalNB()
classifier.fit(X,y)
accuracy(classifier.predict(X),y)

0.8128201760147117

### 这里就是把预测结果提交,看看成绩.

In [8]:
submission=pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
submission['target']=test_pred
submission.to_csv("submission.csv",index=None)
submission.head()

,id,target
0,0,1
1,2,0
2,3,0
3,9,0
4,11,1
